HW 4 -- 

In [86]:
import datetime
import json

import boto3
import requests

In [87]:
USERNAME = "zsuzsi"

In [88]:
# Try different dates to see how the data changes
DATE_PARAM = "2025-11-14"

date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")

# Construct the API URL
url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/{date.strftime('%Y/%m/%d')}"
print(f"Requesting REST API URL: {url}")

# Make the API request
wiki_server_response = requests.get(url, headers={"User-Agent": "curl/7.68.0"})
wiki_response_status = wiki_server_response.status_code
wiki_response_body = wiki_server_response.text

print(f"Wikipedia REST API Response body: {wiki_response_body[:500]}...")
print(f"Wikipedia REST API Response Code: {wiki_response_status}")

# Validate response
if wiki_response_status != 200:
    raise Exception(f"Received non-OK status code from Wiki Server: {wiki_response_status}")
print(f"Successfully retrieved Wikipedia data, content-length: {len(wiki_response_body)}")

Requesting REST API URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/2025/11/14
Wikipedia REST API Response body: {"items":[{"project":"en.wikipedia","access":"all-access","year":"2025","month":"11","day":"14","articles":[{"article":"Main_Page","views":5841654,"rank":1},{"article":"Special:Search","views":872268,"rank":2},{"article":"2025_Bihar_Legislative_Assembly_election","views":823147,"rank":3},{"article":"Google_Chrome","views":300714,"rank":4},{"article":"Election_Commission_of_India","views":260337,"rank":5},{"article":"Wikipedia:Featured_pictures","views":252437,"rank":6},{"article":"Bihar_Legislat...
Wikipedia REST API Response Code: 200
Successfully retrieved Wikipedia data, content-length: 57742


In [89]:
# Parse the API response and extract top edits
wiki_response_parsed = wiki_server_response.json()
top_views = wiki_response_parsed["items"][0]["articles"]

# Transform to JSON Lines format
current_time = datetime.datetime.now(datetime.timezone.utc)
json_lines = ""
for page in top_views[:6]:
    record = {
        "title": page["article"],
        "views": page["views"],
        "rank": page["rank"],
        "date": date.strftime("%Y-%m-%d"),
        "retrieved_at": current_time.replace(tzinfo=None).isoformat(),
    }
    json_lines += json.dumps(record) + "\n"

print(f"Transformed {len(top_views)} records to JSON Lines")
print(f"First few lines:\n{json_lines[:500]}...")

Transformed 1000 records to JSON Lines
First few lines:
{"title": "Main_Page", "views": 5841654, "rank": 1, "date": "2025-11-14", "retrieved_at": "2025-12-10T19:22:48.641031"}
{"title": "Special:Search", "views": 872268, "rank": 2, "date": "2025-11-14", "retrieved_at": "2025-12-10T19:22:48.641031"}
{"title": "2025_Bihar_Legislative_Assembly_election", "views": 823147, "rank": 3, "date": "2025-11-14", "retrieved_at": "2025-12-10T19:22:48.641031"}
{"title": "Google_Chrome", "views": 300714, "rank": 4, "date": "2025-11-14", "retrieved_at": "2025-12-10T1...


In [90]:
S3_WIKI_BUCKET = "zsuzsi-wikidata"
s3 = boto3.client("s3")

bucket_names = [bucket["Name"] for bucket in s3.list_buckets()["Buckets"]]
if S3_WIKI_BUCKET not in bucket_names:
    s3.create_bucket(
        Bucket=S3_WIKI_BUCKET,
        CreateBucketConfiguration={"LocationConstraint": "eu-west-1"},
    )
    print(f"Created new bucket: {S3_WIKI_BUCKET}")
else:
    print(f"Using existing bucket: {S3_WIKI_BUCKET}")

Using existing bucket: zsuzsi-wikidata


In [91]:
#uploading to S3
s3_key = f"raw-views/raw-views-{date.strftime("%Y-%m-%d")}.json"
s3.put_object(
    Bucket=S3_WIKI_BUCKET,
    Key=s3_key,
    Body=json_lines,
)
print(f"Uploaded {len(top_views)} records to s3://{S3_WIKI_BUCKET}/{s3_key}")

expected_key = f"raw-views/raw-views-{date.strftime("%Y-%m-%d")}.json"
try:
    s3.head_object(Bucket=S3_WIKI_BUCKET, Key=expected_key)
    print(f"File uploaded successfully to s3://{S3_WIKI_BUCKET}/{expected_key}")
except Exception as e:
    print(f"File not found at s3://{S3_WIKI_BUCKET}/{expected_key}")
    raise

Uploaded 1000 records to s3://zsuzsi-wikidata/raw-views/raw-views-2025-11-14.json
File uploaded successfully to s3://zsuzsi-wikidata/raw-views/raw-views-2025-11-14.json
